In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [34]:
# Scrape the odds data from the cricket line guru app
def scrape_odds_data():
    response = requests.get("https://www.cricketlineguru.com/odds/")
    soup = BeautifulSoup(response.content, "html.parser")

    odds_data = []

    for match in soup.find_all("div", class_="match-row"):
        team1_name = match.find("span", class_="team1-name").text
        team2_name = match.find("span", class_="team2-name").text
        odds1 = float(match.find("span", class_="odds1").text)
        odds2 = float(match.find("span", class_="odds2").text)

        odds_data.append([team1_name, team2_name, odds1, odds2])

    return odds_data

In [35]:
# Clean and pre-process the odds data
def clean_and_preprocess_odds_data(odds_data):
    odds_data = pd.DataFrame(odds_data, columns=["team1_name", "team2_name", "odds1", "odds2"])
    odds_data = odds_data.dropna()

    return odds_data


In [36]:
# Engineer features from the odds data
def engineer_features(odds_data):
    odds_data["odds_ratio"] = odds_data["odds1"] / odds_data["odds2"]

    return odds_data


In [37]:
# Train a machine learning model to predict the probability of each team winning
def train_model(odds_data):
    X = odds_data[["odds_ratio"]]
    y = odds_data["team1_name"]

    model = LogisticRegression()
    model.fit(X, y)

    return model


In [48]:
# Evaluate the performance of the trained model on a held-out test set
def evaluate_model(model, odds_data):
    X_test = odds_data[["odds_ratio"]]
    y_test = odds_data["team1_name"]

    y_pred = model.predict(X_test)

    accuracy = np.mean(y_pred == y_test)

    print("Accuracy:", accuracy)


In [39]:
# Deploy the trained model to production
def deploy_model(model):
    # Save the model to a file
    with open("odds_prediction_model.pkl", "wb") as f:
        pickle.dump(model, f)


In [54]:
# Use the model to predict the right time to buy and sell a bet
def predict_right_time_to_bet(model, odds_data):
    X = odds_data[["odds_ratio"]]

    y_pred = model.predict(X)

    # If the model predicts that the team with the lower odds is likely to win, then it is a good time to buy a bet on that team.
    # If the model predicts that the team with the higher odds is likely to win, then it is a good time to sell a bet on that team.

    return y_pred
